In [1]:
import csv
import numpy as np
import pandas as pd
import json


import os

In [2]:
with open('/raid/xiaoyuz1/goemotions/goemotions/data/emotions.txt', 'r') as f:
    emotion_list = f.readlines()
    emotion_list = [s.strip() for s in emotion_list]

emotion_text_to_label = dict(zip(emotion_list, range(len(emotion_list))))
label_to_emotion_text = dict(zip(range(len(emotion_list)), emotion_list))

In [15]:
label_to_emotion_text[4]

'approval'

In [16]:
emotion_text_to_label

{'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'optimism': 20,
 'pride': 21,
 'realization': 22,
 'relief': 23,
 'remorse': 24,
 'sadness': 25,
 'surprise': 26,
 'neutral': 27}

In [3]:
!ls /raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/EI-oc/development

2018-EI-oc-En-anger-dev.txt  2018-EI-oc-En-joy-dev.txt
2018-EI-oc-En-fear-dev.txt   2018-EI-oc-En-sadness-dev.txt


In [4]:
intensity_fdr = "/raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/EI-oc"
sub_fdrs = ["development", "test-gold", "training"]


all_new_rows = {}
acc = 0 
for fdr in sub_fdrs:
    for file in os.listdir(os.path.join(intensity_fdr, fdr)):
        path = os.path.join(intensity_fdr, fdr, file)
        df = pd.read_csv(path, sep='\t')

        for i in range(len(df)):
            row = df.iloc[i]
            
            level = row["Intensity Class"].split(":")[0]
            if int(level) == 0:
                continue 

            row_label = "{}_{}".format(row["Affect Dimension"], level)
            new_row = [row["Tweet"], row_label]
            all_new_rows[acc] = new_row

            acc += 1

In [5]:
df_new = pd.DataFrame.from_dict(all_new_rows, orient='index', columns=['text', 'label'])
df_new.reset_index(drop=True, inplace=True)

In [6]:
df_new["label"].unique()

array(['fear_2', 'fear_3', 'fear_1', 'joy_1', 'joy_3', 'joy_2', 'anger_1',
       'anger_2', 'anger_3', 'sadness_2', 'sadness_3', 'sadness_1'],
      dtype=object)

In [7]:
new_path = "/raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/EI-oc/EIoc_goemotions_train.csv"
df_new.to_csv(new_path, index=False)

In [3]:
def process_other_data(path, new_path, emotion_text_to_label):
    df = pd.read_csv(path, sep='\t')
    all_new_rows = {}
    acc = 0
    for i in range(len(df)):
        row = df.iloc[i]
        flag = False
        row_text = None
        new_row = []
        for k,v in row.items():
            if(k == "Tweet"):
                row_text = v
                continue

            if k in emotion_text_to_label and v==1:
                flag = True
                new_row.append(str(emotion_text_to_label[k]))
        if(not flag):
            continue

        all_new_rows[acc] = [row_text, ",".join(new_row)]
        acc += 1

    df_new = pd.DataFrame.from_dict(all_new_rows, orient='index', columns=['text', 'label'])
    df_new.reset_index(drop=True, inplace=True)
    df_new.to_csv(new_path, index=False)
    
    return df_new

In [6]:
path = '/raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/E-c/2018-E-c-En-train.txt'
parent_dir = os.path.abspath(os.path.join(path, os.pardir))
new_path = os.path.join(parent_dir, 'goemotions_train.csv')

path = '/raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/E-c/2018-E-c-En-dev.txt'
parent_dir = os.path.abspath(os.path.join(path, os.pardir))
new_path = os.path.join(parent_dir, 'goemotions_dev.csv')

path = '/raid/xiaoyuz1/goemotions/SemEval2018-Task1-all-data/English/E-c/2018-E-c-En-test-gold.txt'
parent_dir = os.path.abspath(os.path.join(path, os.pardir))
new_path = os.path.join(parent_dir, 'goemotions_test.csv')

process_other_data(path, new_path, emotion_text_to_label)

,text,label
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,"2,20"
1,"Academy of Sciences, eschews the normally sobe...",11
2,I blew that opportunity -__- #mad,"2,11,25"
3,This time in 2 weeks I will be 30... 😥,"17,25"
4,#Deppression is real. Partners w/ #depressed p...,"14,25"
...,...,...
3151,I need a sparkling bodysuit . No occasion. Jus...,17
3152,@everybodyzzmama I've finished reading it; sim...,"11,14,25"
3153,shaft abrasions from panties merely shifted to...,2
3154,All this fake outrage. Y'all need to stop 🤣,"2,11"


In [2]:
# tsv -> csv
df = pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/test.tsv",sep='\t', header=None)
df.to_csv('/raid/xiaoyuz1/goemotions/goemotions/data/test_pred.csv', columns = [0,1], index=False)
pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/test_pred.csv")

In [3]:
with open('/raid/xiaoyuz1/goemotions/goemotions/data/emotions.txt', 'r') as f:
    emotion_list = f.readlines()
    emotion_list = [s.strip() for s in emotion_list]

emotion_text_to_label = dict(zip(emotion_list, range(len(emotion_list))))
label_to_emotion_text = dict(zip(range(len(emotion_list)), emotion_list))

In [4]:
def remap_df(text_arr, label_arr, xx_to_idx, go_to_xx, label_to_emotion_text, out_fname):
    
    xx_label_arr = []
    for labs in label_arr:
        lab_list = labs.split(',')
        xx_idxs = []
        for lab in lab_list:
            go_emotion_text_label = label_to_emotion_text[int(lab)]
            xx_idx = xx_to_idx[go_to_xx[go_emotion_text_label]]
            
            xx_idxs.append(str(xx_idx))
        
        xx_idxs = list(set(xx_idxs))
        xx_lab = ",".join(xx_idxs)
        
        xx_label_arr.append(xx_lab)
        
    df_xx = pd.DataFrame({'text': list(text_arr), 'label': xx_label_arr})
    df_xx.to_csv(out_fname, sep=',', index=False)


## Remap to Ekman’s taxonomy.

In [4]:
import json

ekman_to_go = json.load(open('/raid/xiaoyuz1/goemotions/goemotions/data/ekman_mapping.json'))
go_to_ekman = {'neutral' : 'neutral'}
for k,v in ekman_to_go.items():
    for vi in v:
        go_to_ekman[vi] = k

ekman_texts = list(ekman_to_go.keys())
ekman_to_idx = dict(zip(ekman_texts, range(len(ekman_texts))))
ekman_to_idx["neutral"] = len(ekman_texts)

print("Ekman: ", ekman_to_idx)

Ekman:  {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5, 'neutral': 6}


In [5]:
# print(label_to_emotion_text[0])
# print(go_to_ekman[label_to_emotion_text[0]])

In [8]:
roots = ['train', 'dev', 'test']
for root in roots:
    df = pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/{}.csv".format(root))
    ekman_fname = '/raid/xiaoyuz1/goemotions/goemotions/data/{}_ekman.csv'.format(root)
    xx_label_arr = remap_df(
        df.text.to_numpy(), 
        df.label.to_numpy(), 
        ekman_to_idx, 
        go_to_ekman, 
        label_to_emotion_text,
        ekman_fname,
    )

In [12]:
label2idx = meta['labels2idxes'][0][0]
label2idx = {int(key) : int(value) for key, value in label2idx.items()}
idx2label = {value: key for key, value in label2idx.items()}

## Remap to sentiment

In [13]:
sentiment_to_go = json.load(open('/raid/xiaoyuz1/goemotions/goemotions/data/sentiment_mapping.json'))
go_to_sentiment = {'neutral' : 'neutral'}
for k,v in sentiment_to_go.items():
    for vi in v:
        go_to_sentiment[vi] = k
sentiment_texts = list(sentiment_to_go.keys())
sentiment_to_idx = dict(zip(sentiment_texts, range(len(sentiment_texts))))
sentiment_to_idx["neutral"] = len(sentiment_texts)

print("Sentiment: ", sentiment_to_idx)

Sentiment:  {'positive': 0, 'negative': 1, 'ambiguous': 2, 'neutral': 3}


In [14]:
roots = ['train', 'dev', 'test']
for root in roots:
    df = pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/{}.csv".format(root))
    sentiment_fname = '/raid/xiaoyuz1/goemotions/goemotions/data/{}_sentiment.csv'.format(root)
    
    xx_label_arr = remap_df(
        df.text.to_numpy(), 
        df.label.to_numpy(), 
        sentiment_to_idx, 
        go_to_sentiment, 
        label_to_emotion_text,
        sentiment_fname,
    )

## Remap to FSJ

> we subsample the existing “all”
dataset to select only data points originally labeled
with fear, joy, or sadness, for a final set of 4,136
data points (3,342 of which are the train set).

In [10]:
def filter_df(text_arr, label_arr, emotion_texts_to_include, out_fname):
    
    filtered_label_arr = []
    filtered_text_arr = []
    for text_one, labs in zip(text_arr, label_arr):
        lab_list = labs.split(',')
        xx_idxs = []
        for lab in lab_list:
            go_emotion_text_label = label_to_emotion_text[int(lab)]
            if go_emotion_text_label in emotion_texts_to_include:
                xx_idxs.append(lab)
        
        if len(xx_idxs) < 1:
            continue
        xx_idxs = list(set(xx_idxs))
        xx_lab = ",".join(xx_idxs)
        
        filtered_label_arr.append(xx_lab)
        filtered_text_arr.append(text_one)
        
    df_xx = pd.DataFrame({'text': list(filtered_text_arr), 'label': filtered_label_arr})
    df_xx.to_csv(out_fname, sep=',', index=False)


In [11]:
FEAR_LABEL_STR = str(emotion_text_to_label['fear'])
SADNESS_LABEL_STR = str(emotion_text_to_label['sadness'])
JOY_LABEL_STR = str(emotion_text_to_label['joy'])

print("fear: ", FEAR_LABEL_STR)
print("sadness: ", SADNESS_LABEL_STR)
print("joy: ", JOY_LABEL_STR)

fear:  14
sadness:  25
joy:  17


In [12]:
roots = ['train', 'dev', 'test']
for root in roots:
    df = pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/{}.csv".format(root))
    fsj_fname = '/raid/xiaoyuz1/goemotions/goemotions/data/{}_fsj.csv'.format(root)
    
    fsj_label_arr = filter_df(
        df.text.to_numpy(), 
        df.label.to_numpy(), 
        ["fear","sadness","joy"],
        fsj_fname,
    )

In [6]:
acc_train = 0
for ele in pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/train.csv")['label'].to_numpy():
    if FEAR_LABEL_STR in ele or SADNESS_LABEL_STR in ele or JOY_LABEL_STR in ele:        
        acc_train += 1
print("In train, FSJ count: ", acc_train)

In train, FSJ count:  3342


In [7]:
acc_dev = 0
for ele in pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/dev.csv")['label'].to_numpy():
    if FEAR_LABEL_STR in ele or SADNESS_LABEL_STR in ele or JOY_LABEL_STR in ele:        
        acc_dev += 1
print("In dev, FSJ count: ", acc_dev)

In dev, FSJ count:  401


In [8]:
acc_test = 0
for ele in pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/test.csv")['label'].to_numpy():
    if FEAR_LABEL_STR in ele or SADNESS_LABEL_STR in ele or JOY_LABEL_STR in ele:        
        acc_test += 1
print("In test, FSJ count: ", acc_test)

In test, FSJ count:  393


In [9]:
print("Total, FSJ count: ", acc_train+acc_dev+acc_test)

Total, FSJ count:  4136


In [9]:
from absl import flags


FLAGS = flags.FLAGS
flags.DEFINE_string("test_data", None, "Test tsv file with true labels.")
flags.DEFINE_string("predictions", None, "Predictions tsv file.")
flags.DEFINE_string("output", "results.json", "Output json file.")
flags.DEFINE_string("emotion_file", "data/emotions.txt",
                    "File containing list of emotions.")
flags.DEFINE_boolean("add_neutral", True, "Whether to add neutral as emotion.")
flags.DEFINE_float("threshold", 0.3, "Threshold for binarizing predictions.")



In [26]:

# preds = pd.read_csv(FLAGS.predictions, sep="\t")
# true = pd.read_csv(
#   FLAGS.test_data, sep="\t", header=None, names=["text", "labels", "id"])
# emotions = open(FLAGS.emotion_file).read().splitlines()
# if FLAGS.add_neutral:
# emotions.append("neutral")
num_emotions = 28

# idx2emotion = {i: e for i, e in enumerate(emotions)}

# preds_mat = np.zeros((len(preds), num_emotions))
# true_mat = np.zeros((len(preds), num_emotions))
# for i in range(len(preds)):
# true_labels = [int(idx) for idx in true.loc[i, "labels"].split(",")]

# for j in range(num_emotions):
#     preds_mat[i, j] = preds.loc[i, idx2emotion[j]]
#     true_mat[i, j] = 1 if j in true_labels else 0

true_mat = np.asarray(all_gt)
preds_mat = np.asarray(all_pred)

# threshold = FLAGS.threshold
pred_ind = preds_mat.copy()
# pred_ind[pred_ind > threshold] = 1
# pred_ind[pred_ind <= threshold] = 0
results = {}

results["accuracy"] = skmetric.accuracy_score(true_mat, pred_ind)

results["macro_precision"], results["macro_recall"], results[
  "macro_f1"], _ = skmetric.precision_recall_fscore_support(
      true_mat, pred_ind, average="macro")

results["micro_precision"], results["micro_recall"], results[
  "micro_f1"], _ = skmetric.precision_recall_fscore_support(
      true_mat, pred_ind, average="micro")

results["weighted_precision"], results["weighted_recall"], results[
  "weighted_f1"], _ = skmetric.precision_recall_fscore_support(
      true_mat, pred_ind, average="weighted")

for i in range(num_emotions):
    emotion = emotion_list[int(idx2label[i])]
    emotion_true = true_mat[:, i]
    emotion_pred = pred_ind[:, i]
    results[emotion + "_accuracy"] = skmetric.accuracy_score(emotion_true, emotion_pred)
    results[emotion + "_precision"], results[emotion + "_recall"], results[
        emotion + "_f1"], _ = skmetric.precision_recall_fscore_support(
            emotion_true, emotion_pred, average="binary")
